# DineSafe Toronto: Exploratory Data Analysis (EDA)

This notebook explores restaurant inspection data from the City of Toronto's DineSafe program.

## Data Download Instructions

Before running this notebook, ensure the latest DineSafe dataset is available in `data/raw/`. To download the dataset:

```bash
python src/download_data.py
```

This will:
* Fetch metadata for the package from Toronto's Open Data portal
* Automatically find the latest available resources
* Save the CSV with a timestamped filename to `data/raw/`

Once the data is saved, this notebook will automatically detect and load the most recent file.

## Load the latest raw DineSafe CSV Data


(generated using the `download_data.py` script)

In [12]:
import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
RAW_DIR = PROJECT_ROOT / "data" / "raw"

csv_files = list(RAW_DIR.glob("dinesafe_*.csv")) # finds all files matching this pattern

if not csv_files:
    raise FileNotFoundError(f"No raw DineSave CSV files found in {RAW_DIR.resolve()}") # .resolve() shows the absolute path

latest_file = max(csv_files, key=lambda f: f.stat().st_mtime) # sort by last modified time, then pick the latest

print(f"Loading {latest_file.name}")
df = pd.read_csv(latest_file)

Loading dinesafe_20250606_120907.csv
